## Strategy backtesting notebook
The following notebook is used to backtest the strategy and generate results to be analyzed later. 

### Env variables and imports

In [10]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os.path
import sys
import uuid
import copy
import json
from itertools import product, combinations
import datetime
from datetime import timedelta
import json

import requests
import tqdm
import backtrader as bt
import pyfolio
import pandas as pd
import numpy as np
from matplotlib import pyplot, warnings
from dateutil.relativedelta import relativedelta
from dateutil import rrule
import seaborn as sns
import dask.dataframe as dd

warnings.filterwarnings("ignore")  # Avoid some noise

/home/narboom23/anaconda3/envs/licenta/lib/python3.8/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [11]:
TIMEFRAME = "daily"
# TIMEFRAME = "weekly"

# FRAMEWORK = 'dask'
FRAMEWORK = "pandas"

# PERC_CHANGE_METHOD = 'mean'
PERC_CHANGE_METHOD = "order aware"

FIRST_DATE = datetime.datetime(2000, 1, 1)
LAST_DATE = datetime.datetime(2020, 12, 31)

BASE_DIR = "/home/narboom23/Projects/licenta"
FORWARD_OPT_RESULTS_JSON = f"{BASE_DIR}/results/{TIMEFRAME}/forward_opt_results.json"
BACKWARD_OPT_RESULTS_JSON = f"{BASE_DIR}/results/{TIMEFRAME}/backward_opt_results.json"
PARSED_STRATEGIES_CSV = f"{BASE_DIR}/results/{TIMEFRAME}/parsed_strategies.csv"

In [12]:
TICKER_FULL_LIST = {
    "MMM": ("3M Company", "1976-08-09"),
    "ABT": ("Abbott Laboratories", "1964-03-31"),
    "ADBE": ("Adobe Inc.", "1997-05-05"),
    "AMD": ("Advanced Micro Devices", ""),
    "AES": ("AES Corp", "1998-10-02"),
    "APD": ("Air Products & Chemicals", "1985-04-30"),
    "ALL": ("Allstate Corp", "1995-07-13"),
    "MO": ("Altria Group Inc", "1957-03-04"),
    "AEE": ("Ameren Corp", "1991-09-19"),
    "AEP": ("American Electric Power", "1957-03-04"),
    "AXP": ("American Express", "1976-06-30"),
    "AIG": ("American International Group", "1980-03-31"),
    "AMGN": ("Amgen Inc.", "1992-01-02"),
    "AON": ("Aon plc", "1996-04-23"),
    "APA": ("APA Corporation", "1997-07-28"),
    "AAPL": ("Apple Inc.", "1982-11-30"),
    "AMAT": ("Applied Materials Inc.", "1995-03-16"),
    "ADM": ("Archer-Daniels-Midland Co", "1981-07-29"),
    "T": ("AT&T Inc.", "1983-11-30 (1957-03-04)"),
    "ADSK": ("Autodesk Inc.", "1989-12-01"),
    "ADP": ("Automatic Data Processing", "1981-03-31"),
    "AZO": ("AutoZone Inc", "1997-01-02"),
    "AVY": ("Avery Dennison Corp", "1987-12-31"),
    "BLL": ("Ball Corp", "1984-10-31"),
    "BAC": ("Bank of America Corp", "1976-06-30"),
    "BK": ("The Bank of New York Mellon", "1995-03-31"),
    "BAX": ("Baxter International Inc.", "1972-09-30"),
    "BDX": ("Becton Dickinson", "1972-09-30"),
    "BA": ("Boeing Company", "1957-03-04"),
    "BSX": ("Boston Scientific", "1995-02-24"),
    "BMY": ("Bristol-Myers Squibb", "1957-03-04"),
    "CPB": ("Campbell Soup", "1957-03-04"),
    "COF": ("Capital One Financial", "1998-07-01"),
    "CAH": ("Cardinal Health Inc.", "1997-05-27"),
    "CCL": ("Carnival Corp.", "1998-12-22"),
    "CAT": ("Caterpillar Inc.", "1957-03-04"),
    "CNP": ("CenterPoint Energy", "1985-07-31"),
    "SCHW": ("Charles Schwab Corporation", "1997-06-02"),
    "CVX": ("Chevron Corp.", "1957-03-04"),
    "CI": ("Cigna", "1976-06-30"),
    "CINF": ("Cincinnati Financial", "1997-12-18"),
    "CSCO": ("Cisco Systems", "1993-12-01"),
    "C": ("Citigroup Inc.", "1988-05-31"),
    "CLX": ("The Clorox Company", "1969-03-31"),
    "KO": ("Coca-Cola Company", "1957-03-04"),
    "CL": ("Colgate-Palmolive", "1957-03-04"),
    "CMA": ("Comerica Inc.", "1995-12-01"),
    "CAG": ("Conagra Brands", "1983-08-31"),
    "COP": ("ConocoPhillips", "1957-03-04"),
    "ED": ("Consolidated Edison", ""),
    "GLW": ("Corning Inc.", ""),
    "COST": ("Costco Wholesale Corp.", "1993-10-01"),
    "CSX": ("CSX Corp.", "1967-09-30"),
    "CMI": ("Cummins Inc.", "1965-03-31"),
    "CVS": ("CVS Health", "1957-03-04"),
    "DHR": ("Danaher Corp.", ""),
    "DRI": ("Darden Restaurants", ""),
    "DE": ("Deere & Co.", "1957-03-04"),
    "D": ("Dominion Energy", ""),
    "DOV": ("Dover Corporation", "1985-10-31"),
    "DTE": ("DTE Energy Co.", "1957-03-04"),
    "DUK": ("Duke Energy", "1976-06-30"),
    "EMN": ("Eastman Chemical", "1994-01-01"),
    "ETN": ("Eaton Corporation", ""),
    "ECL": ("Ecolab Inc.", "1989-01-31"),
    "EIX": ("Edison Int'l", "1957-03-04"),
    "EMR": ("Emerson Electric Company", "1965-03-31"),
    "ETR": ("Entergy Corp.", "1957-03-04"),
    "EFX": ("Equifax Inc.", "1997-06-19"),
    "ES": ("Eversource Energy", ""),
    "EXC": ("Exelon Corp.", "1957-03-04"),
    "XOM": ("Exxon Mobil Corp.", "1957-03-04"),
    "FDX": ("FedEx Corporation", "1980-12-31"),
    "FITB": ("Fifth Third Bancorp", ""),
    "FE": ("FirstEnergy Corp", ""),
    "F": ("Ford Motor Company", "1957-03-04"),
    "BEN": ("Franklin Resources", ""),
    "FCX": ("Freeport-McMoRan Inc.", ""),
    "GPS": ("Gap Inc.", "1986-08-31"),
    "GD": ("General Dynamics", "1957-03-04"),
    "GE": ("General Electric", ""),
    "GIS": ("General Mills", "1969-03-31"),
    "GPC": ("Genuine Parts", "1973-12-31"),
    "GL": ("Globe Life Inc.", "1989-04-30"),
    "GWW": ("Grainger (W.W.) Inc.", "1981-06-30"),
    "HAL": ("Halliburton Co.", "1957-03-04"),
    "HIG": ("Hartford Financial Svc.Gp.", "1957-03-04"),
    "HAS": ("Hasbro Inc.", "1984-09-30"),
    "HSY": ("The Hershey Company", "1957-03-04"),
    "HES": ("Hess Corporation", "1984-05-31"),
    "HD": ("Home Depot", "1988-03-31"),
    "HON": ("Honeywell Int'l Inc.", "1964-03-31"),
    "HWM": ("Howmet Aerospace", "1964-03-31"),
    "HPQ": ("HP Inc.", "1974-12-31"),
    "HUM": ("Humana Inc.", ""),
    "HBAN": ("Huntington Bancshares", ""),
    "ITW": ("Illinois Tool Works", "1986-02-28"),
    "INTC": ("Intel Corp.", "1976-12-31"),
    "IBM": ("International Business Machines", "1957-03-04"),
    "IP": ("International Paper", "1957-03-04"),
    "IPG": ("Interpublic Group", "1992-10-01"),
    "IFF": ("International Flavors & Fragrances", "1976-03-31"),
    "JNJ": ("Johnson & Johnson", "1973-06-30"),
    "JPM": ("JPMorgan Chase & Co.", "1975-06-30"),
    "K": ("Kellogg Co.", ""),
    "KEY": ("KeyCorp", "1994-03-01"),
    "KMB": ("Kimberly-Clark", "1957-03-04"),
    "KLAC": ("KLA Corporation", ""),
    "KR": ("Kroger Co.", "1957-03-04"),
    "LB": ("L Brands Inc.", "1983-09-30"),
    "LEG": ("Leggett & Platt", ""),
    "LLY": ("Lilly (Eli) & Co.", "1970-12-31"),
    "LNC": ("Lincoln National", "1976-06-30"),
    "LIN": ("Linde plc", "1992-07-01"),
    "LMT": ("Lockheed Martin Corp.", "1984-07-31"),
    "L": ("Loews Corp.", ""),
    "LOW": ("Lowe's Cos.", "1984-02-29"),
    "MRO": ("Marathon Oil Corp.", "1991-05-01"),
    "MAR": ("Marriott International", ""),
    "MMC": ("Marsh & McLennan", "1987-08-31"),
    "MAS": ("Masco Corp.", "1981-06-30"),
    "MKC": ("McCormick & Co.", ""),
    "MCD": ("McDonald's Corp.", "1970-06-30"),
    "MCK": ("McKesson Corp.", ""),
    "MDT": ("Medtronic plc", "1986-10-31"),
    "MRK": ("Merck & Co.", "1957-03-04"),
    "MU": ("Micron Technology", "1994-09-27"),
    "MSFT": ("Microsoft Corp.", "1994-06-01"),
    "TAP": ("Molson Coors Beverage Company", "1976-06-30"),
    "MCO": ("Moody's Corp", ""),
    "MS": ("Morgan Stanley", ""),
    "MSI": ("Motorola Solutions Inc.", ""),
    "NWL": ("Newell Brands", "1989-04-30"),
    "NEM": ("Newmont Corporation", "1969-06-30"),
    "NEE": ("NextEra Energy", "1976-06-30"),
    "NKE": ("Nike, Inc.", "1988-11-30"),
    "NI": ("NiSource Inc.", ""),
    "NSC": ("Norfolk Southern Corp.", "1957-03-04"),
    "NTRS": ("Northern Trust Corp.", ""),
    "NOC": ("Northrop Grumman", "1985-06-30"),
    "NUE": ("Nucor Corp.", "1985-04-30"),
    "OXY": ("Occidental Petroleum", "1982-12-31"),
    "OMC": ("Omnicom Group", ""),
    "ORCL": ("Oracle Corp.", "1989-08-31"),
    "PCAR": ("Paccar", "1980-12-31"),
    "PH": ("Parker-Hannifin", "1985-11-30"),
    "PAYX": ("Paychex Inc.", ""),
    "PEP": ("PepsiCo Inc.", "1957-03-04"),
    "PKI": ("PerkinElmer", "1985-05-31"),
    "PFE": ("Pfizer Inc.", "1957-03-04"),
    "PNW": ("Pinnacle West Capital", ""),
    "PNC": ("PNC Financial Services", "1988-04-30"),
    "PPG": ("PPG Industries", "1957-03-04"),
    "PPL": ("PPL Corp.", ""),
    "PG": ("Procter & Gamble", "1957-03-04"),
    "PGR": ("Progressive Corp.", "1997-08-04"),
    "PEG": ("Public Service Enterprise Group (PSEG)", "1957-03-04"),
    "PHM": ("PulteGroup", "1984-04-30"),
    "QCOM": ("Qualcomm", ""),
    "RTX": ("Raytheon Technologies", ""),
    "RF": ("Regions Financial Corp.", "1998-08-28"),
    "ROK": ("Rockwell Automation Inc.", ""),
    "SPGI": ("S&P Global Inc.", ""),
    "SLB": ("Schlumberger Ltd.", "1965-03-31"),
    "SHW": ("Sherwin-Williams", "1964-06-30"),
    "SNA": ("Snap-on", "1982-09-30"),
    "SO": ("Southern Company", "1957-03-04"),
    "LUV": ("Southwest Airlines", "1994-07-01"),
    "SWK": ("Stanley Black & Decker", "1982-09-30"),
    "SBUX": ("Starbucks Corp.", ""),
    "STT": ("State Street Corp.", ""),
    "TGT": ("Target Corp.", "1976-12-31"),
    "TXN": ("Texas Instruments", ""),
    "TXT": ("Textron Inc.", "1978-12-31"),
    "TJX": ("TJX Companies Inc.", "1985-09-30"),
    "TFC": ("Truist Financial", "1997-12-04"),
    "TSN": ("Tyson Foods", ""),
    "USB": ("U.S. Bancorp", ""),
    "UNP": ("Union Pacific Corp", "1957-03-04"),
    "UNH": ("UnitedHealth Group Inc.", "1994-07-01"),
    "UNM": ("Unum Group", "1994-03-01"),
    "VLO": ("Valero Energy", ""),
    "VZ": ("Verizon Communications", "1983-11-30"),
    "VFC": ("VF Corporation", "1979-06-30"),
    "VNO": ("Vornado Realty Trust", ""),
    "WMT": ("Walmart", "1982-08-31"),
    "WBA": ("Walgreens Boots Alliance", "1979-12-31"),
    "DIS": ("The Walt Disney Company", "1976-06-30"),
    "WM": ("Waste Management Inc.", ""),
    "WAT": ("Waters Corporation", ""),
    "WFC": ("Wells Fargo", "1976-06-30"),
    "WY": ("Weyerhaeuser", ""),
    "WHR": ("Whirlpool Corp.", ""),
    "WMB": ("Williams Companies", "1975-03-31"),
    "XEL": ("Xcel Energy Inc", "1957-03-04"),
    "YUM": ("Yum! Brands Inc", "1997-10-06"),
}
TICKERS = list(TICKER_FULL_LIST.keys())
TICKERS

['MMM',
 'ABT',
 'ADBE',
 'AMD',
 'AES',
 'APD',
 'ALL',
 'MO',
 'AEE',
 'AEP',
 'AXP',
 'AIG',
 'AMGN',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'ADM',
 'T',
 'ADSK',
 'ADP',
 'AZO',
 'AVY',
 'BLL',
 'BAC',
 'BK',
 'BAX',
 'BDX',
 'BA',
 'BSX',
 'BMY',
 'CPB',
 'COF',
 'CAH',
 'CCL',
 'CAT',
 'CNP',
 'SCHW',
 'CVX',
 'CI',
 'CINF',
 'CSCO',
 'C',
 'CLX',
 'KO',
 'CL',
 'CMA',
 'CAG',
 'COP',
 'ED',
 'GLW',
 'COST',
 'CSX',
 'CMI',
 'CVS',
 'DHR',
 'DRI',
 'DE',
 'D',
 'DOV',
 'DTE',
 'DUK',
 'EMN',
 'ETN',
 'ECL',
 'EIX',
 'EMR',
 'ETR',
 'EFX',
 'ES',
 'EXC',
 'XOM',
 'FDX',
 'FITB',
 'FE',
 'F',
 'BEN',
 'FCX',
 'GPS',
 'GD',
 'GE',
 'GIS',
 'GPC',
 'GL',
 'GWW',
 'HAL',
 'HIG',
 'HAS',
 'HSY',
 'HES',
 'HD',
 'HON',
 'HWM',
 'HPQ',
 'HUM',
 'HBAN',
 'ITW',
 'INTC',
 'IBM',
 'IP',
 'IPG',
 'IFF',
 'JNJ',
 'JPM',
 'K',
 'KEY',
 'KMB',
 'KLAC',
 'KR',
 'LB',
 'LEG',
 'LLY',
 'LNC',
 'LIN',
 'LMT',
 'L',
 'LOW',
 'MRO',
 'MAR',
 'MMC',
 'MAS',
 'MKC',
 'MCD',
 'MCK',
 'MDT',
 'MRK',
 'MU',
 

In [13]:
DEFAULT_FROM_DATE = datetime.datetime(2000, 1, 1)
DEFAULT_TO_DATE = datetime.datetime(2020, 12, 31)
DEFAULT_CASH = 100.0
DEFAULT_COMMISION = 0.0
DEFAULT_CPU_COUNT = 4
DEFAULT_TICKER_LIST = TICKERS

### Utilities

In [14]:
def get_forward_opt_json():
    with open(FORWARD_OPT_RESULTS_JSON, "r") as f:
        return json.loads(f.read())


FORWARD_OPT_JSON = get_forward_opt_json()


def get_backward_opt_json():
    with open(BACKWARD_OPT_RESULTS_JSON, "r") as f:
        return json.loads(f.read())


BACKWARD_OPT_JSON = get_backward_opt_json()


def is_pair_parsed(key, range_val, func_val):
    if key not in BACKWARD_OPT_JSON:
        return False

    if range_val not in BACKWARD_OPT_JSON[key]:
        return False

    if func_val not in BACKWARD_OPT_JSON[key][range_val]:
        return False

    return True

In [15]:
def get_ticker_csv_path(ticker_name, timeframe):
    return f"{BASE_DIR}/data/{timeframe}_tickers/{ticker_name}.csv"

In [16]:
def get_forward_optimal_params(timeframe, date, period, func):
    def get_forward_opt_json():
        with open(f"{BASE_DIR}/results/{timeframe}/forward_opt_results.json", "r") as f:
            return json.loads(f.read())

    FORWARD_OPT_JSON = get_forward_opt_json()
    optimal_args = FORWARD_OPT_JSON[date][period][func]

    return optimal_args


def get_backward_optimal_params(
    timeframe, date, period, ffunc, bfunc, bfunc_type, bfunc_barrier_type
):
    def get_backward_opt_json():
        with open(
            f"{BASE_DIR}/results/{timeframe}/backward_opt_results.json", "r"
        ) as f:
            return json.loads(f.read())

    backward_OPT_JSON = get_backward_opt_json()
    optimal_args = backward_OPT_JSON[date][period][ffunc]

    if optimal_args is None or bfunc == "None":
        return None

    return optimal_args[bfunc][bfunc_type][bfunc_barrier_type]

In [17]:
import csv


def get_parsed_configs():
    def iterate_parsed_csv():
        with open(PARSED_STRATEGIES_CSV, "r") as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=",")
            line_count = 0
            for row in csv_reader:
                yield row

    res = []

    for row in iterate_parsed_csv():
        row[0] = int(row[0])
        res.append(row)

    return res


def add_row_to_csv(new_row):
    data = get_parsed_configs()

    with open(PARSED_STRATEGIES_CSV, "w") as out:
        csv_out = csv.writer(out)
        csv_out.writerow(new_row)

        for row in data:
            csv_out.writerow(row)

In [18]:
perc_allocated = [25, 20, 15, 10, 5, 3]

timeframes = ["weekly", "daily"]
range_values = [
    "6-month",
    "12-month",
]
forward_func_values = ["mean", "order"]
backward_func_values = [
    "mean",
    "order",
    "None",
]
backward_func_options = [
    "single",
]

backward_opt_func_barrier_types = ["upper", "lower", "best"]

In [19]:
from itertools import product

all_options = list(
    product(
        perc_allocated,
        timeframes,
        range_values,
        forward_func_values,
        backward_func_values,
        backward_func_options,
        backward_opt_func_barrier_types,
    )
)
len(all_options)

432

In [20]:
unparsed_configs = [x for x in all_options if list(x) not in get_parsed_configs()]
print(len(unparsed_configs))
(
    PERC_CHANGE,
    TIMEFRAME,
    PERIOD,
    FFUNC,
    BFUNC,
    BFUNC_TYPE,
    BFUNC_BARRIER_TYPE,
) = unparsed_configs[0]
PERC_CHANGE, TIMEFRAME, PERIOD, FFUNC, BFUNC, BFUNC_TYPE, BFUNC_BARRIER_TYPE

215


(25, 'weekly', '6-month', 'mean', 'mean', 'single', 'lower')

In [21]:
FILE_KEY = f"{PERC_CHANGE}_{TIMEFRAME}_{PERIOD}_{FFUNC}_{BFUNC}_{BFUNC_TYPE}_{BFUNC_BARRIER_TYPE}"
FILE_KEY

'25_weekly_6-month_mean_mean_single_lower'

### Strategy
The main strategy class used

In [26]:
class RaynerTeoStrategy(bt.Strategy):
    params = (
        ("timeframe", "daily"),
        ("period", "max"),
        ("forward_opt_func", "mean"),
        ("backward_opt_func", "mean"),
        ("backward_opt_func_type", "single"),
        ("backward_opt_func_barrier_type", "upper"),
    )

    def __init__(self):
        self.inds = dict()
        for i, d in enumerate(self.datas):
            self.inds[d] = dict()

            if self.params.timeframe == "daily":
                self.inds[d]["sma180"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=180
                )
                self.inds[d]["sma200"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=200
                )
                self.inds[d]["sma220"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=220
                )
                self.inds[d]["sma240"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=240
                )
                self.inds[d]["sma260"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=260
                )
                self.inds[d]["sma280"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=280
                )

                self.inds[d]["rsi2"] = bt.indicators.RSI(
                    d.close, period=2, safediv=True
                )
                self.inds[d]["rsi4"] = bt.indicators.RSI(
                    d.close, period=4, safediv=True
                )
                self.inds[d]["rsi6"] = bt.indicators.RSI(
                    d.close, period=6, safediv=True
                )
                self.inds[d]["rsi8"] = bt.indicators.RSI(
                    d.close, period=8, safediv=True
                )
                self.inds[d]["rsi10"] = bt.indicators.RSI(
                    d.close, period=10, safediv=True
                )
                self.inds[d]["rsi12"] = bt.indicators.RSI(
                    d.close, period=12, safediv=True
                )
                self.inds[d]["rsi14"] = bt.indicators.RSI(
                    d.close, period=14, safediv=True
                )

                self.inds[d]["adx8"] = bt.indicators.ADX(d, period=8)
                self.inds[d]["adx16"] = bt.indicators.ADX(d, period=16)
                self.inds[d]["adx32"] = bt.indicators.ADX(d, period=32)

                self.inds[d]["ppo8"] = bt.indicators.PPO(
                    d.close, period1=8, period2=220
                )
                self.inds[d]["ppo16"] = bt.indicators.PPO(
                    d.close, period1=16, period2=220
                )
                self.inds[d]["ppo32"] = bt.indicators.PPO(
                    d.close, period1=32, period2=220
                )

                self.inds[d]["stochastic8"] = bt.indicators.Stochastic(
                    d, period=8, safediv=True
                )
                self.inds[d]["stochastic16"] = bt.indicators.Stochastic(
                    d, period=16, safediv=True
                )
                self.inds[d]["stochastic32"] = bt.indicators.Stochastic(
                    d, period=32, safediv=True
                )
            elif self.params.timeframe == "weekly":
                self.inds[d]["sma30"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=30
                )
                self.inds[d]["sma35"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=35
                )
                self.inds[d]["sma40"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=40
                )
                self.inds[d]["sma45"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=45
                )
                self.inds[d]["sma50"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=50
                )
                self.inds[d]["sma55"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=55
                )

                self.inds[d]["rsi2"] = bt.indicators.RSI(
                    d.close, period=2, safediv=True
                )
                self.inds[d]["rsi4"] = bt.indicators.RSI(
                    d.close, period=4, safediv=True
                )
                self.inds[d]["rsi6"] = bt.indicators.RSI(
                    d.close, period=6, safediv=True
                )

                self.inds[d]["adx3"] = bt.indicators.ADX(d, period=3)
                self.inds[d]["adx6"] = bt.indicators.ADX(d, period=6)
                self.inds[d]["adx9"] = bt.indicators.ADX(d, period=9)

                self.inds[d]["ppo3"] = bt.indicators.PPO(d.close, period1=3, period2=30)
                self.inds[d]["ppo6"] = bt.indicators.PPO(d.close, period1=6, period2=30)
                self.inds[d]["ppo9"] = bt.indicators.PPO(d.close, period1=9, period2=30)

                self.inds[d]["stochastic3"] = bt.indicators.Stochastic(
                    d, period=3, safediv=True
                )
                self.inds[d]["stochastic6"] = bt.indicators.Stochastic(
                    d, period=6, safediv=True
                )
                self.inds[d]["stochastic9"] = bt.indicators.Stochastic(
                    d, period=9, safediv=True
                )

            self.inds[d]["order_placed_days_ago"] = 0

    def next(self):
        year, month = self.datetime.date().year, self.datetime.date().month
        month = month - 1
        if month == 0:
            month = 12
            year = year - 1

        # The optimal month will be 'yy-mm' where these are the just passed month & it's year
        month = f"0{month}" if month < 10 else str(month)
        year = str(year)[-2:]
        optimal_year_month_key = f"{year}-{month}"

        optimal_params = get_forward_optimal_params(
            self.p.timeframe,
            optimal_year_month_key,
            self.p.period,
            self.p.forward_opt_func,
        )
        optimal_sma = f'sma{optimal_params["maperiod"]}'
        optimal_rsi_open = f'rsi{optimal_params["rsi_open_period"]}'
        optimal_days_ago = optimal_params["days_ago_close_period"]

        backward_optimal_params = get_backward_optimal_params(
            self.p.timeframe,
            optimal_year_month_key,
            self.p.period,
            self.p.forward_opt_func,
            self.p.backward_opt_func,
            self.p.backward_opt_func_type,
            self.p.backward_opt_func_barrier_type,
        )

        for i, d in enumerate(self.datas):
            dt, dn = self.datetime.date(), d._name
            pos = self.getposition(d).size

            if not pos:
                if (
                    d.close[0] > self.inds[d][optimal_sma][0]
                    and self.inds[d][optimal_rsi_open][0] < 30
                ):
                    self.buy(data=d)
            else:
                if (
                    self.params.backward_opt_func == "None"
                    or backward_optimal_params is None
                ):
                    if (
                        self.inds[d][optimal_rsi_open][0] >= 30
                        or self.inds[d]["order_placed_days_ago"] >= optimal_days_ago
                    ):
                        self.sell(data=d)
                        self.inds[d]["order_placed_days_ago"] = 0

                    else:
                        self.inds[d]["order_placed_days_ago"] += 1
                else:
                    barrier = backward_optimal_params["barrier"]
                    indicator = backward_optimal_params["indicator"]

                    if self.params.backward_opt_func_barrier_type == "lower":
                        new_condition = self.inds[d][indicator] <= barrier
                    elif self.params.backward_opt_func_barrier_type == "upper":
                        new_condition = self.inds[d][indicator] >= barrier
                    elif self.params.backward_opt_func_barrier_type == "best":
                        best_type = backward_optimal_params["type"]

                        if best_type == "lower":
                            new_condition = self.inds[d][indicator] <= barrier
                        elif best_type == "upper":
                            new_condition = self.inds[d][indicator] >= barrier

                    if new_condition is True:
                        if (
                            self.inds[d][optimal_rsi_open][0] >= 30
                            or self.inds[d]["order_placed_days_ago"] >= optimal_days_ago
                        ):
                            self.sell(data=d)
                            self.inds[d]["order_placed_days_ago"] = 0
                        else:
                            self.inds[d]["order_placed_days_ago"] += 1

                    else:
                        self.inds[d]["order_placed_days_ago"] += 1

## Run configuration

In [27]:
%%time
# Create a cerebro entity
cerebro = bt.Cerebro(stdstats=False)

cerebro.addobserver(bt.observers.Broker)
cerebro.addobserver(bt.observers.Trades)


# Add a strategy
strats = cerebro.addstrategy(
    RaynerTeoStrategy,
    **{
        "timeframe": TIMEFRAME,
        "period": PERIOD,
        "forward_opt_func": FFUNC,
        "backward_opt_func": BFUNC,
        "backward_opt_func_type": BFUNC_TYPE,
        "backward_opt_func_barrier_type": BFUNC_BARRIER_TYPE,
    },
)

# Add data feeds
datalist = [
    (get_ticker_csv_path(ticker_name, TIMEFRAME), ticker_name)
    for ticker_name in DEFAULT_TICKER_LIST
]

for i in range(len(DEFAULT_TICKER_LIST)):
    data = bt.feeds.YahooFinanceCSVData(
        dataname=datalist[i][0],
        name=datalist[i][1],
        fromdate=DEFAULT_FROM_DATE,
        todate=DEFAULT_TO_DATE,
        reverse=False,
    )
    data.plotinfo.plot = False
    cerebro.adddata(data)

# Setup and run
cerebro.broker.setcash(DEFAULT_CASH)
cerebro.addanalyzer(bt.analyzers.PyFolio, _name="pyfolio")
cerebro.addsizer(bt.sizers.PercentSizer, percents=PERC_CHANGE)
cerebro.broker.setcommission(commission=DEFAULT_COMMISION)
strat = cerebro.run(maxcpus=None)  # None uses all available CPUs

CPU times: user 52.3 s, sys: 831 ms, total: 53.2 s
Wall time: 53.2 s


In [28]:
pyfoliozer = strat[0].analyzers.getbyname("pyfolio")
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()

### SAVE THE RESULTS

In [25]:
FINAL_RESULTS_PATH = f"{BASE_DIR}/results/final_results.csv"


def save_returns(returns):
    new_returns = pyfolio.timeseries.perf_stats(
        returns
    )  # save this in main df along with key
    new_returns.name = FILE_KEY

    existing_df = pd.read_csv(FINAL_RESULTS_PATH, index_col=0)
    existing_df = existing_df.append(new_returns)

    existing_df.to_csv(FINAL_RESULTS_PATH)


save_returns(returns)

In [26]:
# a = returns # USE THIS TO GENERATE THE DATAFRAME LATER IF NEEDED
# pd.DataFrame([a.values], columns=a.index, index=[FILE_KEY]).to_csv(f"results/final_results.csv")

In [28]:
def save_extra(pd_obj, file_key, thing):
    file_path = f"{BASE_DIR}/results/{TIMEFRAME}/extra/{thing}/{file_key}_{thing}.csv"
    pd_obj.to_csv(file_path)


for pd_obj, thing in [
    (returns, "returns"),
    (positions, "positions"),
    (transactions, "transactions"),
    (gross_lev, "gross_lev"),
]:
    save_extra(pd_obj, FILE_KEY, thing)

In [22]:
add_row_to_csv(
    [PERC_CHANGE, TIMEFRAME, PERIOD, FFUNC, BFUNC, BFUNC_TYPE, BFUNC_BARRIER_TYPE]
)

In [26]:
pd.read_csv(FINAL_RESULTS_PATH, index_col=0)

,Annual return,Cumulative returns,Annual volatility,Sharpe ratio,Calmar ratio,Stability,Max drawdown,Omega ratio,Sortino ratio,Skew,Kurtosis,Tail ratio,Daily value at risk
15_weekly_6-month_mean_mean_single_lower,0.344509,2.619427,0.219961,1.456613,1.373258,0.961804,-0.250870,1.389391,2.169347,-0.113978,10.961415,1.258123,-0.026441
25_weekly_6-month_mean_mean_single_upper,0.401866,3.339844,0.248006,1.487917,1.289264,0.922687,-0.311702,1.384368,2.139051,-0.802160,9.132713,1.210247,-0.029781
